<a href="https://colab.research.google.com/github/dongjaeseo/CS2705/blob/main/lung_cancer_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# 0, 1 masked image -> finding centroid
def centroid(mask):
    row_sum = 0
    column_sum = 0
    count = 0
    for i, row in enumerate(mask):
        for j, pixel in enumerate(row):
            if pixel == 1:
                row_sum += i
                column_sum += j
                count += 1

    if count == 0:
        raise TypeError("Wrong image input: Only masked image of 0 and 1 is allowed")

    return column_sum/count, row_sum/count

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask1.png', 0)
image1 = image1/ 255.

x, y = centroid(image1)
print(image1.shape)

plt.imshow(image1, cmap = 'gray')

print(x, y)
plt.plot(x, y, marker="o", markersize=5, markeredgecolor="red", markerfacecolor="green")
plt.show()

In [ ]:
def find_boundary(mask):
    boundary = []
    for i, row in enumerate(mask):
        for j, pixel in enumerate(row):
            try:
                if mask[i-1][j] < pixel:
                    boundary.append((j,i-0.5))
            except:
                boundary.append((j, i-0.5))
            
            try:
                if mask[i][j+1] < pixel:
                    boundary.append((j+0.5, i))
            except:
                boundary.append((j+0.5, i))
            
            try:
                if mask[i+1][j] < pixel:
                    boundary.append((j,i+0.5))
            except:
                boundary.append((j, i+0.5))

            try:
                if mask[i][j-1] < pixel:
                    boundary.append((j-0.5, i))
            except:
                boundary.append((j-0.5, i))

    return boundary

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask1.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

boundary = find_boundary(image1)

plt.imshow(image1, cmap = 'gray')
for i in boundary:
    plt.plot(i[0], i[1], marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
plt.show()